In [1]:
n_cards = 20

In [49]:
import random
from collections import Counter

import numpy as np
import pandas as pd


def simulate_draws(remaining_cards, sample_size, num_draws=5_000_000):
    remaining_cards_array = np.array(remaining_cards)
    sums = []
    for i in range(num_draws):
        sums.append(sum(np.random.choice(remaining_cards_array, sample_size, replace=False)))
    return sums

def probability_distribution(drawn_cards):
    # Define the full deck (1 to 13 for each suit)
    full_deck = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] * 4

    # Remove the drawn cards from the full deck
    for card in drawn_cards:
        full_deck.remove(card)

    # Simulate draws to get the distribution of sums
    sums = simulate_draws(full_deck, sample_size=n_cards - len(drawn_cards))

    # Count the occurrences of each sum
    counter = Counter(sums)

    # Convert to probability distribution
    total = sum(counter.values())
    distribution = {k: v / total for k, v in counter.items()}
    return distribution



In [50]:
def price(cards_drawn_so_far, strikes):
    dist = probability_distribution(cards_drawn_so_far)
    sum_so_far = sum(cards_drawn_so_far)
    instruments = []
    instruments.append(
        {
            "kind": "Future",
            "strike": np.nan,
            "price": sum((k + sum_so_far) * p for k, p in dist.items()),
            "delta": 1.0,
        }
    )
    for strike in strikes:
        instruments.append(
            {
                "kind": "Call",
                "strike": strike,
                "price": sum(
                    max(k + sum_so_far - strike, 0) * p for k, p in dist.items()
                ),
                "delta": sum(p for k, p in dist.items() if k + sum_so_far >= strike),
            }
        )
        instruments.append(
            {
                "kind": f"Put",
                "strike": strike,
                "price": sum(
                    max(strike - (k + sum_so_far), 0) * p for k, p in dist.items()
                ),
                "delta": -sum(p for k, p in dist.items() if k + sum_so_far <= strike),
            }
        )
    return pd.DataFrame(instruments).set_index(["strike", "kind"]).sort_index().round(2)

In [47]:
price(
    [6, 3, 13, 13, 11, 9, 12, 5, 12,6,6,1,11,5,5,1,2,4,9, 8],
    strikes=[130, 140, 150],
)

price  delta
strike kind                
130.0  Call     12.0    1.0
       Put       0.0    0.0
140.0  Call      2.0    1.0
       Put       0.0    0.0
150.0  Call      0.0    0.0
       Put       8.0   -1.0
NaN    Future  142.0    1.0

In [51]:
price(
    [],
    strikes=[130, 150],
)

price  delta
strike kind                 
130.0  Call     11.74   0.78
       Put       1.73  -0.24
150.0  Call      1.73   0.24
       Put      11.72  -0.78
NaN    Future  140.01   1.00